# Simulating Deletions

In [1]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = cobra.test.create_test_model("textbook")
ecoli_model = cobra.test.create_test_model("ecoli")

## Knocking out single genes and reactions

A commonly asked question when analyzing metabolic models is what will happen if a certain reaction was not allowed to have any flux at all. This can tested using cobrapy by

In [2]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x7f41bb363550>
pfk knocked out:  <Solution 0.704 at 0x7f41bb363710>


For evaluating genetic manipulation strategies, it is more interesting to examine what happens if given genes are knocked out as doing so can affect no reactions in case of redundancy, or more reactions if gene when is participating in more than one reaction.

In [3]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x7f41bb35bf60>
pfkA knocked out:  <Solution 0.874 at 0x7f41bb35bd68>
pfkB knocked out:  <Solution 0.704 at 0x7f41bb35bf98>


## Single Deletions

Perform all single gene deletions on a model

In [4]:
deletion_results = single_gene_deletion(cobra_model)

These can also be done for only a subset of genes

In [5]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

,growth,status
ids,,
(b0356),0.873922,optimal
(b0351),0.873922,optimal
(b1849),0.873922,optimal
(b2296),0.873922,optimal
(b2587),0.873922,optimal
(b0726),0.858307,optimal
(b1276),0.873922,optimal
(b3115),0.873922,optimal
(b1478),0.873922,optimal


This can also be done for reactions

In [6]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

,growth,status
ids,,
(Biomass_Ecoli_core),0.000000e+00,optimal
(ETOHt2r),8.739215e-01,optimal
(ATPM),9.166475e-01,optimal
(ACALD),8.739215e-01,optimal
(EX_ac_e),8.739215e-01,optimal
(ALCD2x),8.739215e-01,optimal
(ACt2r),8.739215e-01,optimal
(ACALDt),8.739215e-01,optimal
(ACONTb),8.988837e-16,optimal


## Double Deletions

Double deletions run in a similar way.

In [7]:
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:]).round(4)

,growth,status
ids,,
"(b2465, b2464)",0.8739,optimal
(b3919),0.7040,optimal
(b2935),0.8739,optimal
"(b2935, b0008)",0.8739,optimal
(b2464),0.8739,optimal
"(b0008, b2464)",0.8739,optimal
"(b3919, b2464)",0.7040,optimal
"(b2935, b3919)",0.7040,optimal
"(b2465, b3919)",0.7040,optimal


By default, the double deletion function will automatically use multiprocessing, splitting the task over up to 4 cores if they are available. The number of cores can be manually specified as well. Setting use of a single core will disable use of the multiprocessing library, which often aids debugging.

In [8]:
start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=2)
t1 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 2 cores" % t1)

start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=1)
t2 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 1 core" % t2)

print("Speedup of %.2fx" % (t2 / t1))

Double gene deletions for 200 genes completed in 2.53 sec with 2 cores
Double gene deletions for 200 genes completed in 4.09 sec with 1 core
Speedup of 1.62x


Double deletions can also be run for reactions.

In [9]:
double_reaction_deletion(
    cobra_model, cobra_model.reactions[2:7]).round(4)

,growth,status
ids,,
(ACt2r),0.8739,optimal
"(ACONTa, ACONTb)",0.0000,optimal
(ACONTb),0.0000,optimal
"(ADK1, ACONTa)",0.0000,optimal
(ADK1),0.8739,optimal
"(ACKr, ACt2r)",0.8739,optimal
(ACONTa),0.0000,optimal
"(ADK1, ACONTb)",0.0000,optimal
(ACKr),0.8739,optimal
